(2-2) 사용자 정의 데이터셋 생성

In [1]:
import pandas as pd
import numpy as np

In [2]:
### 데이터 준비
filename = '../data/iris.csv'
irisDF = pd.read_csv(filename)
irisDF.head(3)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


In [3]:
irisNP = np.loadtxt(filename, delimiter=',', skiprows=1, usecols=[0, 1, 2, 3])
irisNP[:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [4]:
### 데이터의 타입 체크
irisDF.__class__.__name__, irisNP.__class__.__name__

('DataFrame', 'ndarray')

In [5]:
if irisDF.__class__.__name__ == 'DataFrame':
    print('DF')
else:
    print('-----')

DF


In [6]:
### 데이터의 타입 체크
isinstance(irisDF, pd.DataFrame), isinstance(irisNP, pd.DataFrame)

(True, False)

In [7]:
isinstance([10], list), isinstance({10}, list)

(True, False)

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [9]:
### 사용자정의 Dataset 클래스
class DLDataset(Dataset):
    
    # 초기화 함수
    def __init__(self, x_data, y_data):
        super().__init__()
        
        # x, y 데이터가 DF라면 ndarray로 바꾸고, ndarray라면 그대로 저장
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data
        
        # ndarray -> tensor
        self.feature = torch.FloatTensor(x_data)     # Float
        self.target = torch.LongTensor(y_data)       # Long
    
    # 데이터셋의 개수 체크 함수 콜백함수 (callback function)
    def __len__(self):
        return self.target.shape[0]
    
    # 특정 인덱스 데이터 + 라벨 반환 콜백함수 (callback function)
    def __getitem__(self, index):
        return self.feature[index], self.target[index]

(2-2) 데이터셋 인스턴스 생성

In [10]:
### 피처와 라벨로 분리
featureDF = irisDF[irisDF.columns[:-1]]
targetSR = irisDF[irisDF.columns[-1]]

In [11]:
### object 타입 타겟 -> int 타입 타겟 변환
from sklearn.preprocessing import LabelEncoder

targetNP = LabelEncoder().fit_transform(targetSR)
targetNP = targetNP.reshape(-1, 1)      # 1차원 -> 2차원으로 변환

In [12]:
# 데이터셋 생성 => DF, NP
my_dataset = DLDataset(featureDF, targetNP)

In [13]:
my_dataset[0], featureDF.iloc[0], targetNP[0]

((tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0])),
 sepal_length    5.1
 sepal_width     3.5
 petal_length    1.4
 petal_width     0.2
 Name: 0, dtype: float64,
 array([0]))

In [14]:
# 데이터셋 생성 => NP, NP
my_dataset2 = DLDataset(irisNP, targetNP)

In [15]:
my_dataset2[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0]))

(2-3) 학습용, 검증용, 테스트용 Dataset

In [16]:
### 파이토치
from torch.utils.data import random_split

# 학습용, 검증용, 테스트 데이터 비율
trainDS, validDS, testDS = random_split(my_dataset2, [0.7, 0.1, 0.2], generator=torch.Generator().manual_seed(42))

In [17]:
type(trainDS)

torch.utils.data.dataset.Subset

In [18]:
print(f'(train) Subset 속성=>\nindices : {trainDS.indices}\ndataset : {trainDS.dataset}')

(train) Subset 속성=>
indices : [42, 95, 30, 64, 52, 35, 130, 40, 82, 17, 108, 94, 68, 97, 117, 127, 41, 44, 57, 140, 149, 32, 23, 102, 16, 113, 71, 18, 67, 66, 0, 25, 101, 112, 91, 3, 59, 116, 86, 84, 106, 142, 43, 39, 26, 98, 93, 20, 87, 19, 120, 114, 7, 63, 76, 89, 36, 45, 37, 56, 58, 122, 51, 145, 24, 21, 105, 62, 15, 11, 48, 133, 88, 50, 6, 134, 111, 8, 49, 75, 69, 124, 4, 147, 80, 100, 99, 141, 47, 107, 13, 109, 129, 28, 38, 53, 121, 5, 55, 31, 73, 74, 54, 29, 12]
dataset : <__main__.DLDataset object at 0x000001FAF00810A0>


In [19]:
len(trainDS), len(validDS), len(testDS)

(105, 15, 30)

(3) DataLoader 생성 : 학습용, 검증용, 테스트용

In [20]:
# DataLoader 생성
# drop_last 매개변수 : 배치사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 (기본 : False)
batch_size = 10
trainDL = DataLoader(trainDS, batch_size=batch_size, drop_last=True)
validDL = DataLoader(validDS, batch_size=batch_size)
testDL = DataLoader(testDS, batch_size=batch_size)

len(trainDL), len(validDL), len(testDL)

(10, 2, 3)

In [21]:
# Epoch 당 반복 단위
print(f'batch_size: {batch_size}')
print(f'trainDS : {len(trainDS)}, validDS : {len(validDS)}, testDS : {len(testDS)}')
print(f'trainDL : {len(trainDL)}, validDS : {len(validDL)}, testDS : {len(testDL)}')

batch_size: 10
trainDS : 105, validDS : 15, testDS : 30
trainDL : 10, validDS : 2, testDS : 3


In [22]:
# DataLoader 
for _, (feature, target) in enumerate(trainDL):
    print(f'[{_}] feature {feature.shape} target {target.shape}')

[0] feature torch.Size([10, 4]) target torch.Size([10, 1])
[1] feature torch.Size([10, 4]) target torch.Size([10, 1])
[2] feature torch.Size([10, 4]) target torch.Size([10, 1])
[3] feature torch.Size([10, 4]) target torch.Size([10, 1])
[4] feature torch.Size([10, 4]) target torch.Size([10, 1])
[5] feature torch.Size([10, 4]) target torch.Size([10, 1])
[6] feature torch.Size([10, 4]) target torch.Size([10, 1])
[7] feature torch.Size([10, 4]) target torch.Size([10, 1])
[8] feature torch.Size([10, 4]) target torch.Size([10, 1])
[9] feature torch.Size([10, 4]) target torch.Size([10, 1])


(4) Model 클래스 정의 : 입/출력 피쳐수, 층 수, 은닉층의 노드 수
- 구조 설계
    - 입력층 : 입력 <= 피쳐 개수 (iris : 4개)
    - 은닉층 : 마음대로, 알아서 잘
    - 출력층 : 출력 <= [분류] 타겟 클래스 개수 [회귀] 1개

In [23]:
# 모델 클래스 정의
# 클래스명 : CModel
class CModel(torch.nn.Module):
    # 구성요소 정의 함수
    def __init__(self, in_, out_):
        super().__init__()      # 부모 클래스 생성자
        self.layer1 = torch.nn.Linear(in_, 10)
        self.relu = torch.nn.ReLU()
        self.layer2 = torch.nn.Linear(10, 6)
        self.layer3 = torch.nn.Linear(6, out_)
    # 순방향 학습 진행 함수
    def forward(self, x):
        x = self.layer1(x)      # W1X1 + W2X2 + ... + WnXn + b 10개 반환
        x = self.relu(x)        # relu() 결과 10개 반환
        x = self.layer2(x)      # W1X1 + W2X2 + ... + WnXn + b 6개 반환
        x = self.relu(x)        # relu() 결과 6개 반환
        x = self.layer3(x)      # W1X1 + W2X2 + ... + WnXn + b 3개 반환
        return x

(5) 학습 준비 : 실행 디바이스, 모델, 최적화, 손실함수, 학습회수 

In [24]:
## 실행 디바이스 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 학습 횟수
EPOCHS = 50

In [25]:
my_dataset.feature.shape

torch.Size([150, 4])

In [26]:
## 모델 인스턴스 생성
IN, OUT = my_dataset.feature.shape[1], len(np.unique(targetNP))
model = CModel(IN, OUT).to(DEVICE)
print(f'모델 구조 확인 : {model}')

모델 구조 확인 : CModel(
  (layer1): Linear(in_features=4, out_features=10, bias=True)
  (relu): ReLU()
  (layer2): Linear(in_features=10, out_features=6, bias=True)
  (layer3): Linear(in_features=6, out_features=3, bias=True)
)


In [27]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.3762,  0.1638, -0.4266, -0.1546],
        [ 0.0384, -0.2837,  0.2531, -0.3386],
        [ 0.0407,  0.1095,  0.3939, -0.3299],
        [-0.3834, -0.4521, -0.2811, -0.4469],
        [ 0.4007, -0.1101, -0.0933,  0.3541],
        [-0.3032, -0.0756, -0.3729, -0.2430],
        [-0.1014, -0.1632, -0.1527, -0.4893],
        [-0.4517, -0.4799,  0.4458, -0.4033],
        [-0.2505,  0.3246,  0.3947,  0.2691],
        [-0.0715, -0.1714,  0.2324,  0.3386]], requires_grad=True)
Parameter containing:
tensor([ 0.4525, -0.0774, -0.0690,  0.3655,  0.3624,  0.0961,  0.4992, -0.4873,
         0.1587, -0.1773], requires_grad=True)
Parameter containing:
tensor([[ 0.0065, -0.1793,  0.1458, -0.3154, -0.2777, -0.0974, -0.2348, -0.1711,
         -0.2132,  0.2392],
        [-0.2735, -0.1542, -0.0144, -0.2331,  0.0415, -0.2214,  0.0453, -0.1169,
         -0.2853,  0.0128],
        [-0.2015,  0.3035,  0.2511,  0.0629, -0.2224, -0.2040,  0.2282,  0.0339,
         -0.1625, -0.0276],

In [28]:
# 손실함수
LOSS_FN = nn.CrossEntropyLoss().to(DEVICE)

# 최적화 인스턴스
import torch.optim as optim
OPTIMIZER = optim.Adam(model.parameters())

* 학습 및 검증관련 함수 정의

In [35]:
import torchmetrics.functional as metrics
from torchmetrics.classification import Accuracy

preds = torch.randn(10, 5).softmax(dim=-1)
targets = torch.randint(5, (10,))

acc = metrics.accuracy(preds, targets, task='multiclass', num_classes=5)
print(f'ACC : {acc}')

ACC : 0.6000000238418579


In [36]:
targets

tensor([1, 0, 2, 4, 4, 1, 0, 1, 0, 1])

In [49]:
## 학습 진행함수
def training():
    # 학습모드 => 정규화, 경사하강법, 드랍아웃 등의 기능 활성화
    model.train()
    
    # 배치크기만큼 학습 진행 및 저장
    train_loss = []
    for cnt, (x, y) in enumerate(trainDL):
        # 배치크기만큼의 학습 데이터 준비
        feature, target = x.to(DEVICE), y.to(DEVICE)
        target = target.squeeze()
        
        # 학습
        pre_target = model(feature)
        # print(f'pre_target => {pre_target.shape}, {pre_target.ndim}D')
        # print(f'target => {target.shape}, {target.ndim}D')
        
        # 손실계산
        loss = LOSS_FN(pre_target, target)
        train_loss.append(loss)
        # W, b 업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()
    
    # 학습 진행 메시지 출력
    print(f'[Train loss] {loss}')
    # acc = metrics.accuracy(pre_target, target, task='multiclass', num_classes=3)
    # print(f'acc : {acc}')
    
    return train_loss

In [50]:
## 검증 및 평가 진행함수
# 매개변수 dataLoader : 검증 또는 테스트 데이터셋에 대한 Loader
def testing(dataLoader):
    # 추론모드 => 정규화, 경사하강법, 드랍아웃 등의 기능 비활성화
    model.eval()
    
    with torch.no_grad():
        # 배치크기만큼 학습 진행 및 저장
        val_loss = []
        for cnt, (feature, target) in enumerate(dataLoader):
            # 배치크기만큼의 학습 데이터 준비
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            target = target.squeeze()
            
            # 학습
            pre_target = model(feature)
            
            # 손실계산
            loss = LOSS_FN(pre_target, target)
            val_loss.append(loss)
            
            # 배치 단위 학습 진행 메시지 출력
            # print(f'[Train {cnt} batch LOSS] => {loss}')
    
    # 에포크 단위 학습 진행 메시지 출력
    acc = metrics.accuracy(pre_target, target, task='multiclass', num_classes=3)
    print(f'[Valid loss] => {loss} [Valid loss] => {acc}')
    
    return val_loss

In [43]:
## 예측 함수
def predict():
    pass

(6) 학습진행

In [72]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

scheduler = ReduceLROnPlateau(optimizer=OPTIMIZER, mode='min', patience=3)

In [78]:
### 지정된 횟수만큼 처음부터 끝까지 학습 및 검증 진행
### 목표 : 최적(Error 최소화)의 W, b를 가진 모델 완성
for eps in range(1, EPOCHS+1):
    # 학습
    train_loss = training()
    
    # 검증
    val_loss = testing(validDL)
    print(f'[{eps}/{EPOCHS}] TRAIN {sum(train_loss)/len(train_loss)}, VALID {sum(val_loss)/len(val_loss)}\n')
    
    # 조기종료 기능 => 조건 : val_loss가 지정된 횟수(예: 5) 이상 개선이 안 되면 학습 종료
    scheduler.step(val_loss[-1])
    print(scheduler.num_bad_epochs)
    if scheduler.num_bad_epochs >= scheduler.patience:
        print(f'Early stopping at epoch {eps}')
        break

[Train loss] 0.0045134639367461205
[Valid loss] => 0.04841848835349083 [Valid loss] => 1.0
[1/50] TRAIN 0.01001050416380167, VALID 0.02441542036831379

0
[Train loss] 0.0045134639367461205
[Valid loss] => 0.04841848835349083 [Valid loss] => 1.0
[2/50] TRAIN 0.010010505095124245, VALID 0.02441542036831379

1
[Train loss] 0.0045134639367461205
[Valid loss] => 0.04841848835349083 [Valid loss] => 1.0
[3/50] TRAIN 0.010010505095124245, VALID 0.02441542036831379

2
[Train loss] 0.0045134639367461205
[Valid loss] => 0.04841848835349083 [Valid loss] => 1.0
[4/50] TRAIN 0.01001050416380167, VALID 0.02441542036831379

3
Early stopping at epoch 4
